# Phase 2: Setup Agent (Basic) - ReAct Pattern

**Goal**: Implement a basic setup agent using MCP tools and a fixed skeleton.

This notebook implements:
1. MCP integration (6 servers)
2. Fixed skeleton (5 steps)
3. Basic ReAct pattern (agent + tools)

Based on `deep_research_from_scratch/notebooks/2_research_agent.ipynb`


In [1]:
# Environment setup
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2


## MCP Integration

Configure MCP client to connect to our FastMCP servers.


In [ ]:
%%writefile ../src/mcp_md/mcp_integration.py

"""MCP Integration for MD Setup Tools."""

import sys
from pathlib import Path
from langchain_mcp_adapters.client import MultiServerMCPClient

def create_mcp_client() -> MultiServerMCPClient:
    """Create MCP client with all servers configured."""
    python_exe = sys.executable
    
    server_config = {
        "structure": {
            "transport": "stdio",
            "command": python_exe,
            "args": ["-m", "servers.structure_server"]
        },
        "genesis": {
            "transport": "stdio",
            "command": python_exe,
            "args": ["-m", "servers.genesis_server"]
        },
        "complex": {
            "transport": "stdio",
            "command": python_exe,
            "args": ["-m", "servers.complex_server"]
        },
        "ligand": {
            "transport": "stdio",
            "command": python_exe,
            "args": ["-m", "servers.ligand_server"]
        },
        "export": {
            "transport": "stdio",
            "command": python_exe,
            "args": ["-m", "servers.export_server"]
        },
        "qc_min": {
            "transport": "stdio",
            "command": python_exe,
            "args": ["-m", "servers.qc_min_server"]
        }
    }
    
    return MultiServerMCPClient(server_config)

async def load_mcp_tools():
    """Load all MCP tools as a dictionary."""
    client = create_mcp_client()
    tools = await client.get_tools()
    return {tool.name: tool for tool in tools}


Writing ../src/mcp_md/mcp_integration.py


## Setup State

Define state for the setup phase (to be used in Notebooks 2-3).


In [ ]:
%%writefile ../src/mcp_md/state_setup.py

"""State Definitions for Setup Phase."""

import operator
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class SetupState(TypedDict):
    """State for setup phase subgraph."""
    # Input from parent
    simulation_brief: dict
    research_brief: str
    
    # Execution state
    setup_messages: Annotated[Sequence[BaseMessage], add_messages]
    current_step: str
    step_iteration: int
    
    # Output to parent
    outputs: dict
    decision_log: Annotated[list[dict], operator.add]
    raw_notes: Annotated[list[str], operator.add]

class SetupOutputState(TypedDict):
    """Output from setup phase."""
    outputs: dict
    decision_log: Annotated[list[dict], operator.add]
    setup_messages: Annotated[Sequence[BaseMessage], add_messages]
    raw_notes: Annotated[list[str], operator.add]

# Fixed skeleton steps
SETUP_STEPS = [
    "structure_fetch",
    "structure_repair",
    "ligand_param",
    "complex_generation",
    "qc_check"
]


Writing ../src/mcp_md/state_setup.py


## Summary

Notebook 2 basic structure created:
- ✅ MCP integration configured
- ✅ Setup state defined
- ✅ Fixed skeleton established

**Next**: Notebook 3 - Coordinator-Tools pattern
